# Introduction
At the moment, this file only contains commands to fine tune the HQ model, which has shown the best performance (see the linked master thesis in the README.md for more details).

The code can be easily extended to fine tune on one of the other datasets

## Note
You need to set the `OPENAI_API_KEY` environment variable to your OpenAI API Key for this code to work

In [1]:
from openai import OpenAI
import os
import time

In [2]:
api_key = os.environ["OPENAI_API_KEY"]
assert api_key is not None

In [3]:
client = OpenAI(api_key=api_key)

In [5]:
training_data_folder = "training_data_train_test"

files = [f"{training_data_folder}/{file}" for file in os.listdir(training_data_folder)
         if file.startswith("training") and "new" in file]


In [6]:
files

['training_data_train_test/training_FT-CR-new-test.jsonl',
 'training_data_train_test/training_FT-CR-new-train.jsonl',
 'training_data_train_test/training_FT-HQ-new-test.jsonl',
 'training_data_train_test/training_FT-HQ-new-train.jsonl',
 'training_data_train_test/training_FT-XL-new-test.jsonl',
 'training_data_train_test/training_FT-XL-new-train.jsonl']

As we are just retraining lost fine-tunes, simply run the fine-tuning job for HQ alone, as it has the best outcomes from the evaluations

In [7]:
train_hq = "training_data_train_test/training_FT-HQ-new-train.jsonl"
test_hq = "training_data_train_test/training_FT-HQ-new-test.jsonl"

## Run the fine-tuning jobs
Note that you need to execute the cells one by one and wait for each to output before running the next one. Otherwise I had trubbles getting an output

In [12]:
train_file = client.files.create(
    file=open(train_hq, "rb"),
    purpose="fine-tune"
)

test_file = client.files.create(
    file=open(test_hq, "rb"),
    purpose="fine-tune"
)

In [70]:
fine_tuning_job_gpt_3_5_lucas = client.fine_tuning.jobs.create(
    model = "gpt-3.5-turbo",
    training_file=train_file.id,
    validation_file=test_file.id,
    hyperparameters={"batch_size": 2, "learning_rate_multiplier": 0.05, "n_epochs": 1},
    suffix = "FT-HQ-new-lucas"
)

In [69]:
fine_tuning_job_gpt_3_5_default = client.fine_tuning.jobs.create(
    model = "gpt-3.5-turbo",
    training_file=train_file.id,
    validation_file=test_file.id,
    #hyperparameters={"batch_size": 2, "learning_rate_multiplier": 0.05, "n_epochs": 3},
    suffix = "FT-HQ-new-default"
)

### GPT-4 fine-tuning

Code which could be used to fine-tune GPT-4 if we decide to do so. Unfortunatelly the code errors out at the moment as we do not have access to fine-tuning GPT-4 at the moment

In [75]:
fine_tuning_job_gpt_4_lucas = client.fine_tuning.jobs.create(
    model = "gpt-4",
    training_file=train_file.id,
    validation_file=test_file.id,
    hyperparameters={"batch_size": 2, "learning_rate_multiplier": 0.05, "n_epochs": 1},
    suffix = "FT-HQ-new-lucas"
)

BadRequestError: Error code: 400 - {'error': {'message': 'Model gpt-4-0613 is not available for fine-tuning or does not exist.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}

In [ ]:
fine_tuning_job_gpt_4_default = client.fine_tuning.jobs.create(
    model = "gpt-4",
    training_file=train_file.id,
    validation_file=test_file.id,
    #hyperparameters={"batch_size": 2, "learning_rate_multiplier": 0.05, "n_epochs": 3},
    suffix = "FT-HQ-new-default"
)

## Monitor progress

Rerun the following cell to get an update on the current fine-tune job

In [9]:
print(f"3.5 FT-HQ-new-lucas:   {client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_3_5_lucas.id).status}")
print(f"3.5 FT-HQ-new-default: {client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_3_5_default.id).status}")
# print(f"4   FT-HQ-new-lucas:   {client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_4_lucas.id)}")
# print(f"4   FT-HQ-new-default: {client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_4_default.id)}")

NameError: name 'fine_tuning_job_gpt_3_5_lucas' is not defined

## Test the fine-tuned models

In [105]:
# get fine tuned models
gpt3_5_lucas_model = client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_3_5_lucas.id).fine_tuned_model
gpt3_5_default_model = client.fine_tuning.jobs.retrieve(fine_tuning_job_gpt_3_5_default.id).fine_tuned_model


In [5]:
def get_completion(model, prompt: str, temperature: float = 0.3) -> str:
    completion = client.chat.completions.create(
        model = model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        stop="END"
    )

    return completion.choices[0].message.content

In [8]:
get_completion("ft:gpt-3.5-turbo-0613:rg-neuroinformatics:ft-hq-new-default:8sCkSlVq",
              "Question: Hello, this is pizzeria Romano, how can I help you?\nAnswer: Hello, I'd like to make a reservation.\nQuestion: Certainly, when would you like to come by?\nKeywords: Tuesday\nAnswer:\n\n###\n\n"
)

" I'd like to come by on Tuesday. "